In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from amplpy import AMPL, ampl_notebook
from joblib import load, dump
import sys
sys.path.append("../src")
from utils import cost_prediction

import warnings
warnings.filterwarnings("ignore")
pd.set_option("future.no_silent_downcasting", True) # Prevent silent data type changes during operations for future compatibility

Instantiate the AMPL object

In [28]:
ampl = ampl_notebook(
    modules=["highs","coin"], # solver to be used
    license_uuid="d2b35988-195b-44a4-bca2-fd80a770586f" # license key
)

Licensed to AMPL Community Edition License for <hchoi0309@gmail.com>.


In [29]:
%%ampl_eval
reset;

Define the model

In [30]:
%%ampl_eval
reset;  # Clear any previous model

# Sets
set ROUTES;                  # Set of routes
set AIRPLANE_TYPES;          # Set of airplane types

# Parameters
param a {ROUTES} >= 0;           # Base fare price coefficient
param b {ROUTES} >= 0;           # Price elasticity coefficient
param y {ROUTES};               # Yearly effect coefficient
param c {ROUTES} >= 0;          # Cost per seat
param N {ROUTES} >= 0;          # Predicted daily passengers
param C {AIRPLANE_TYPES} >= 0;  # Capacity of each airplane type
param R {AIRPLANE_TYPES} >= 0;  # Total airplanes of each type
param t >= 0, <= 1;            # Target market share
param year >= 0;               # Current year

# Variables
var x {ROUTES} >= 0, integer;     # Daily number of seats sold
var r {ROUTES, AIRPLANE_TYPES} >= 0, integer;  # Number of airplanes assigned

# Objective: Maximize profit
maximize Profit: 
    sum {i in ROUTES} (
        x[i] * (a[i] * exp(-b[i] * x[i] + y[i] * year) - c[i])
    );

# Constraints
subject to DemandLimit {i in ROUTES}:
    x[i] <= t * N[i];

subject to MinCapacity {i in ROUTES}:
    x[i] >= sum {k in AIRPLANE_TYPES} (0.7 * C[k] * r[i,k]);

subject to MaxCapacity {i in ROUTES}:
    x[i] <= sum {k in AIRPLANE_TYPES} (C[k] * r[i,k]);

subject to FleetLimit {k in AIRPLANE_TYPES}:
    sum {i in ROUTES} r[i,k] <= R[k];

In [31]:
allegiant_df = pd.read_csv("../data/allegiant_routes_info_transformed.csv")
allegiant_df["year"] = 2019
allegiant_df["quarter"] = 4
allegiant_df.head()

,distance,airport_1,airport_2,daily_passengers,state_1,city_1,state_2,city_2,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2,year,quarter
0,1342,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4
1,1340,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4
2,587,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4
3,588,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4
4,612,BNA,PIE,42.760000,TN,Nashville,FL,Tampa,1177657.0,555.4,2861173.0,1320.9,36.166687,-86.779932,37.860600,-78.804199,2019,4


In [32]:
allegiant_df["cost"] = cost_prediction(allegiant_df, 2019)
display(allegiant_df.head())

,distance,airport_1,airport_2,daily_passengers,state_1,city_1,state_2,city_2,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2,year,quarter,cost
0,1342,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4,234.94394
1,1340,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4,234.59380
2,587,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4,102.76609
3,588,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4,102.94116
4,612,BNA,PIE,42.760000,TN,Nashville,FL,Tampa,1177657.0,555.4,2861173.0,1320.9,36.166687,-86.779932,37.860600,-78.804199,2019,4,107.14284


In [33]:
preprocessing_pipeline = load("../models/preprocessing_pipeline.joblib")
feature_names = load("../models/feature_names.joblib")
allegiant_transformed_df = preprocessing_pipeline.transform(allegiant_df)
allegiant_transformed_df = pd.DataFrame(allegiant_transformed_df, columns=feature_names)
allegiant_transformed_df.head()

,quarter_1,quarter_2,quarter_3,quarter_4,city_1,city_2,airport_1,airport_2,state_1,state_2,year,distance,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2
0,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.216246,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496
1,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.213401,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496
2,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.857693,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496
3,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.856271,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496
4,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,0.545563,-0.117519,-1.082934,-1.044005,1.068477,1.204085,-0.822132,-0.826930,-0.916193,-0.538597,-0.800943,-0.465810,0.186579,-0.091321,0.745495


In [34]:
demand_model = load("../models/demand_prediction_model.joblib")
allegiant_df["predicted_passengers"] = np.abs(demand_model.predict(allegiant_transformed_df))
allegiant_df["route"] = allegiant_df["airport_1"] + "_" + allegiant_df["airport_2"]
allegiant_df = allegiant_df[["route", "predicted_passengers", "cost"]]
display(allegiant_df.head())

,route,predicted_passengers,cost
0,AUS_IAD,314.190399,234.94394
1,AUS_IAD,314.190399,234.59380
2,BNA_IAD,220.477982,102.76609
3,BNA_IAD,220.477982,102.94116
4,BNA_PIE,55.914410,107.14284


In [35]:
fare_model = load("../models/route_fare_models.joblib")

allegiant_df["a"] = allegiant_df["route"].apply(lambda x: fare_model[x]["a"]).apply(lambda x: max(x, 0))
allegiant_df["b"] = allegiant_df["route"].apply(lambda x: fare_model[x]["b"]).apply(lambda x: max(x, 0))
allegiant_df["y"] = allegiant_df["route"].apply(lambda x: fare_model[x]["y"])

display(allegiant_df.head())

,route,predicted_passengers,cost,a,b,y
0,AUS_IAD,314.190399,234.94394,8.181866e+04,0.00015,-0.002792
1,AUS_IAD,314.190399,234.59380,8.181866e+04,0.00015,-0.002792
2,BNA_IAD,220.477982,102.76609,1.789587e-13,0.00170,0.017371
3,BNA_IAD,220.477982,102.94116,1.789587e-13,0.00170,0.017371
4,BNA_PIE,55.914410,107.14284,1.795778e+90,0.00000,-0.100730


In [36]:
fleet_df = pd.read_csv("../data/allegiant_fleet.csv")
display(fleet_df)

,Aircraft Type,Number of Aircraft,Seats
0,Airbus A319-100,34,156
1,Airbus A320-200 A,14,177
2,Airbus A320-200 B,78,186
3,Boeing 737 MAX 200,2,190


Define the problem

In [26]:
# 2. Load the data into AMPL
# Assuming allegiant_df and fleet_df are your dataframes containing the necessary data
ampl.set["ROUTES"] = allegiant_df["route"].unique()
ampl.set["AIRPLANE_TYPES"] = fleet_df["Aircraft Type"]

# Load parameters
ampl.param["a"] = {row["route"]: row["a"] for _, row in allegiant_df.drop_duplicates("route").iterrows()}
ampl.param["b"] = {row["route"]: row["b"] for _, row in allegiant_df.drop_duplicates("route").iterrows()}
ampl.param["y"] = {row["route"]: row["y"] for _, row in allegiant_df.drop_duplicates("route").iterrows()}
ampl.param["c"] = {row["route"]: row["cost"] for _, row in allegiant_df.drop_duplicates("route").iterrows()}
ampl.param["N"] = {row["route"]: row["predicted_passengers"] for _, row in allegiant_df.drop_duplicates("route").iterrows()}

ampl.param["C"] = {row["Aircraft Type"]: row["Seats"] for _, row in fleet_df.iterrows()}
ampl.param["R"] = {row["Aircraft Type"]: row["Number of Aircraft"] for _, row in fleet_df.iterrows()}

ampl.param["t"] = 0.8  # Target market share (80%)
ampl.param["year"] = 2019  # Current year

# 3. Set solver and solve
ampl.option["solver"] = "ipopt"  # Use ipopt for nonlinear problems
ampl.solve()

# 4. Retrieve and display results
try:
    # Get results
    x_values = ampl.getVariable("x").getValues().toPandas()
    r_values = ampl.getVariable("r").getValues().toPandas()
    profit = ampl.getObjective("Profit").value()

    print("Optimal seat allocation:")
    print(x_values)
    print("\nAircraft assignment:")
    print(r_values)
    print(f"\nTotal profit: ${profit:,.2f}")

except Exception as e:
    print("Error solving the model:")
    print(e)
    
    # Debug information
    print("\nParameter values:")
    print("Routes:", list(ampl.set["ROUTES"]))
    print("N values:", ampl.param["N"].getValues().toPandas())
    print("a values:", ampl.param["a"].getValues().toPandas())

Ipopt 3.12.13: 
==> Warning: Treating 159 binary and 753 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2549
Number of nonzeros in Lagrangian Hessian.............:      108

Total number of variables............................:      912
                     variables with only lower bounds:        0
                variables with lower and upper bounds:   

In [37]:
ampl.set["ROUTES"] = allegiant_df["route"].drop_duplicates()
ampl.param["a"] = allegiant_df.set_index("route")["a"]
ampl.param["b"] = allegiant_df.set_index("route")["b"]
ampl.param["y"] = allegiant_df.set_index("route")["y"]
ampl.param["c"] = allegiant_df.set_index("route")["cost"]
ampl.param["N"] = allegiant_df.set_index("route")["predicted_passengers"]

ampl.set["AIRPLANE_TYPES"] = fleet_df["Aircraft Type"]
ampl.param["C"] = fleet_df.groupby("Aircraft Type")["Seats"].first()
ampl.param["R"] = fleet_df.groupby("Aircraft Type")["Number of Aircraft"].first()

ampl.param["t"] = 0.8
ampl.param["year"] = 2019

In [38]:
%%ampl_eval
option solver highs
solve;

In [39]:
# Retrieve results
x = ampl.getVariable("x").getValues().toPandas()
r_assign = ampl.getVariable("r").getValues().toPandas()
profit = ampl.getObjective("Profit").value()

# Display results
print("Optimal Seat Sales (x):")
print(x)

print("\nAirplane Assignments (r_assign):")
print(r_assign)

print(f"\nTotal Profit: {profit}")

Optimal Seat Sales (x):
         x.val
AUS_BOS      0
AUS_BUR      0
AUS_BWI      0
AUS_CAK      0
AUS_CLE      0
...        ...
SNA_MFR      0
SNA_MSO      0
SRQ_IAD      0
VPS_BWI      0
VPS_IAD      0

[225 rows x 1 columns]

Airplane Assignments (r_assign):
                            r.val
index0  index1                   
AUS_BOS Airbus A319-100         0
        Airbus A320-200 A       0
        Airbus A320-200 B       0
        Boeing 737 MAX 200      0
AUS_BUR Airbus A319-100         0
...                           ...
VPS_BWI Boeing 737 MAX 200      0
VPS_IAD Airbus A319-100         0
        Airbus A320-200 A       0
        Airbus A320-200 B       0
        Boeing 737 MAX 200      0

[900 rows x 1 columns]

Total Profit: 0.0
